# Market-Report.ipynb

weekly などのマーケットコメント用データ収集ノートブック


In [ ]:
%load_ext autoreload
%autoreload 2

import os
import re
import sqlite3
import sys
import warnings

import curl_cffi
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import src.analyze.reporting.market_report_utils as mru
from src.market.alternative import tsa

pd.options.display.precision = 2
from pathlib import Path

import yfinance as yf

warnings.simplefilter("ignore")
from dotenv import load_dotenv

load_dotenv()


QUANTS_DIR = Path(os.environ.get("QUANTS_DIR"))  # type: ignore
DATA_DIR = Path(os.environ.get("DATA_DIR"))  # type: ignore
FRED_DIR = Path(os.environ.get("FRED_DIR"))  # type: ignore
BLOOMBERG_ROOT_DIR = Path(os.environ.get("BLOOMBERG_ROOT_DIR"))  # type: ignore
TSA_DIR = Path(os.environ.get("TSA_DIR"))  # type: ignore

# sys.path.append(str(ROOT_DIR))
sys.path.insert(0, str(QUANTS_DIR))

from src.market.fred import HistoricalCache

from src import us_treasury

fred_db_path = FRED_DIR / "FRED.db"
tsa_db_path = TSA_DIR / "TSA.db"
sp_price_db_path = BLOOMBERG_ROOT_DIR / "SP_Indices_Price.db"

# daily data collect
cache = HistoricalCache()
cache.sync_all_presets()

# bbg = mru.BloombergDataProcessor()
# bbg.delete_table_from_db(db_path=bbg.sp_price_db, table_name="Members")
# bbg.store_sp_indices_price_to_database()
# sp500_tickers = [re.split(" ", s)[0] for s in bbg.get_current_sp_members()]
# display(sp500_tickers)

tsa_data_collector = tsa.TSAPassengerDataCollector()
df = tsa_data_collector.scrape_tsa_passenger_data()
tsa_data_collector.store_to_tsa_database(df=df, db_path=tsa_db_path)

analyzer = mru.MarketPerformanceAnalyzer()

In [ ]:
conn = sqlite3.connect(sp_price_db_path)
df = pd.read_sql("SELECT * FROM Members", con=conn)
display(df)


## S&P Index, Magnificent 7 & SOX Index, Sector

- TODO: 直近 5 日間の分足データをプロットできるようにしておく -> やっぱりいらないかも


In [ ]:
price = analyzer.price_data
display(
    price[[col for col in price.columns if col.startswith("XL")] + ["^SOX"]].tail(6)
)
performance_dict = analyzer.get_performance_groups()

display(mru.apply_df_style(performance_dict["US_and_SP500_Indices"]))
performance_sp_indices = performance_dict["US_and_SP500_Indices"]
# display(performance_sp_indices["last_Tuesday"])
analyzer.plot_sp500_indices()

display(mru.apply_df_style(performance_dict["Mag7_SOX"]))
# performance_mag7_sox = performance_dict["Mag7_SOX"]
# display(performance_mag7_sox[["last_Tuesday"]])
analyzer.plot_mag7_sox()

poerformance_sector = performance_dict["Sector"]
display(mru.apply_df_style(poerformance_sector))
# display(poerformance_sector["last_Tuesday"])
analyzer.plot_sector_performance()

display(mru.apply_df_style(performance_dict["Metals"]))
analyzer.plot_metal()


### SP500 Sector ETF Beta(vs SP500)


In [ ]:
sp500_etf = [col for col in list(analyzer.SECTOR_MAP_EN.keys()) if col.startswith("XL")]
tickers_to_download = sp500_etf + ["^SPX"]
df_price = analyzer.yf_download_with_curl(
    tickers_to_download=tickers_to_download, period="max"
)
df_price = df_price.loc[df_price["variable"] == "Adj Close"].drop(columns=["variable"])

freq = "W"
window_years = 3
df_beta = mru.rolling_beta(
    df_price=df_price, target_col="^SPX", freq=freq, window_years=window_years
)
fig = mru.plot_rolling_beta(
    df_beta=df_beta,
    freq=freq,
    window_years=window_years,
    tickers_to_plot=["XLK", "XLU", "XLRE"],
    target_index_name="S&P500",
)
fig.show()


### (✏️Test) Kalman Filter Beta


In [ ]:
df_beta = mru.calculate_kalman_beta(
    df_price=df_price, target_col="^SPX", freq=freq, window_years=window_years
)
display(df_beta)

fig = mru.plot_rolling_beta(
    df_beta=df_beta,
    freq=freq,
    window_years=window_years,
    tickers_to_plot=["XLK", "XLU", "XLRE"],
    target_index_name="S&P500",
    beta_variable_name="beta_3y_w_kalman_3years",
)
fig.show()


## US Interest Rate / Corporate Bond Spread


In [ ]:
us_treasury.plot_us_interest_rates_and_spread_3(
    db_path=fred_db_path, start_date="2000-01-01"
)
us_treasury.plot_us_corporate_bond_spreads(
    db_path=fred_db_path, json_config_path=FRED_DIR / "fred_series.json"
)


## VIX, High Yield Spread, US Economic Uncertainty Index


In [ ]:
mru.plot_vix_and_high_yield_spread()
mru.plot_vix_and_uncertainty_index()


## Dollars Index vs Metals


### Price and Rolling Correlation


In [ ]:
metal_analyzer = mru.DollarsIndexAndMetalsAnalyzer()
df_metal_price = metal_analyzer.price
fig = metal_analyzer.plot_us_dollar_index_and_metal_price()
fig.show()

df_corr = mru.rolling_correlation(df_price=df_metal_price, target_col="DTWEXAFEGS")

for ticker in ["GLD", "SLV", "CPER", "PPLT", "PALL"]:
    fig = mru.plot_rolling_correlation(
        df_corr=df_corr,
        ticker_to_plot=ticker,
        target_index_name="Nominal Advanced Foreign Economies US Dolalr Index",
    )
    fig.show()


### Beta


In [ ]:
df_beta = mru.rolling_beta(
    df_price=df_metal_price, target_col="DTWEXAFEGS", freq="W", window_years=5
)
fig = mru.plot_rolling_beta(
    df_beta=df_beta,
    freq="W",
    window_years=5,
    tickers_to_plot=["GLD", "SLV", "PPLT"],
    target_index_name="Nominal Advanced Foreign Economies US Dolalr Index",
)
fig.show()


## SP500 Index weekly return


In [ ]:
analyzer = mru.MarketPerformanceAnalyzer()
tickers_sp500 = analyzer.TICKERS_SP500

price = yf.download(tickers=tickers_sp500, auto_adjust=False, period="max")[
    "Close"
].dropna(how="any")
log_return = np.log(price / price.shift(5)).dropna(how="any")
log_return = log_return.loc[log_return.index.year == 2025]
log_return_long = (
    log_return.stack().reset_index().rename(columns={0: "weekly_log_return"})
)
log_return_long["weekly_log_return"] = log_return_long["weekly_log_return"].mul(100)
display(log_return_long)
stats = (
    log_return_long.groupby("Ticker")["weekly_log_return"]
    .agg(["mean", "std"])
    .rename(columns={"mean": "log_return_avg(%)", "std": "log_return_std(%)"})
)
display(stats)


In [ ]:
# FacetGridを初期化し、ティッカーごとにグラフを分割
# col_wrap=2 で横に2つ並べるように指定
g = sns.FacetGrid(log_return_long, col="Ticker", col_wrap=2, height=4, sharex=True)

# 各グラフにヒストグラムをプロット
g.map(
    sns.histplot,
    "weekly_log_return",
    bins=100,
    kde=True,
    stat="density",
    edgecolor="black",
)

# 各サブプロットに平均値と±1シグマの垂直線を追加
for ax in g.axes.flatten():
    ticker = ax.get_title().split("=")[1].strip()

    # 該当ティッカーの統計量を取得
    mean_val = stats.loc[ticker, "log_return_avg(%)"]
    std_val = stats.loc[ticker, "log_return_std(%)"]

    # 平均 (µ) の線
    ax.axvline(
        mean_val,
        color="red",
        linestyle="--",
        linewidth=2,
        label=f"Mean: {mean_val:.4f}",
    )

    # +1シグマ (µ + σ) の線
    ax.axvline(
        mean_val + std_val,
        color="green",
        linestyle=":",
        linewidth=1.5,
        label="+1 Sigma",
    )

    # -1シグマ (µ - σ) の線
    ax.axvline(
        mean_val - std_val,
        color="green",
        linestyle=":",
        linewidth=1.5,
        label="-1 Sigma",
    )

    # 凡例をプロットに追加（スペースがあれば）
    # ax.legend()

# レイアウトの調整
plt.suptitle("Weekly log return", y=1.02, fontsize=16)
g.set_axis_labels("return", "density")
g.tight_layout()
plt.show()


## S&P セクター別 ETF

- SPDR


In [ ]:
session = curl_cffi.Session(impersonate="safari15_5")
ticker_list = " ".join(
    [
        "XLK",
        "XLF",
        "XLV",
        "XLY",
        "XLC",
        "XLI",
        "XLP",
        "XLE",
        "XLU",
        "XLRE",
        "XLB",
        "^SPX",
    ]
)
Tickers_obj = yf.Tickers(ticker_list, session=session)
data = (
    Tickers_obj.history(period="max", interval="1d")
    .stack()
    .reset_index()
    .rename(columns={"Date": "date"})
)
display(data)


In [ ]:
stock = yf.Ticker("MSFT")
earnings_info = stock.calendar
display(earnings_info)
historical_earnings = stock.get_earnings_dates()
display(historical_earnings)
